In [3]:
import requests
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from konlpy.tag import Kkma
from konlpy.tag import Okt

In [4]:
# 참고 사이트 https://yeo0.github.io/data/2018/09/24/5.-%EB%A1%9C%EA%B7%B8%EC%9D%B8%EC%9D%B4-%ED%95%84%EC%9A%94%ED%95%9C-%EC%82%AC%EC%9D%B4%ED%8A%B8%EC%97%90%EC%84%9C%EC%9D%98-%ED%81%AC%EB%A1%A4%EB%A7%81/
# 참고 사이트 2 https://hashcode.co.kr/questions/9084/%EC%9E%A1%ED%94%8C%EB%9E%98%EB%8B%9B-%EB%A1%9C%EA%B7%B8%EC%9D%B8-%ED%9B%84-%EC%8A%A4%ED%81%AC%EB%A0%88%EC%9D%B4%ED%95%91-%EB%AC%B8%EC%9D%98%EB%93%9C%EB%A6%BD%EB%8B%88%EB%8B%A4-%E3%85%9C%E3%85%9C

# 로그인 할 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

In [5]:
def ind_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    page = random.sample(range(1, pages),30)
    reviews = {}
    for i in page: # 페이지를 랜덤으로 하여 300개의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [6]:
# 각 산업군의 기업 리뷰 스크래핑
reviews1 = dict(ind_reviews(1001))
reviews2 = dict(ind_reviews(900))

In [31]:
#특수문자 제거
def sub(dic):
    dic = [re.sub(r"[^가-힣A-Za-z0-9]", " ", str(content)) for content in dic]
    return dic

In [32]:
#리뷰별 키워드 추출 후 중첩 리스트 생성  
def append(text):
    temp=[]
    for i in text:
        temp.append(okt.nouns(i))
    return temp

In [33]:
#산업군 별 키워드 추출 후 리스트 생성
def extend(text):
    temp=[]
    for i in text:
        temp.extend(okt.nouns(i))
    return temp

In [34]:
reviews_1 = sub(reviews1)
reviews_1

['위치 좋고 타이틀 좋고 일도 나름 잘 배울 수 있음 ',
 '칼퇴보장과 비교적 자유로운 점심시간이 좋았습니다  다만 공무원 사회이다보니 보수적인 분위기가 있습니다',
 '편하게 일할수있음 구내 식당도 맛있음 1층카페도 좋음',
 '팀바이팀이지만 비교적 무난하게 다닐 수 있는 곳',
 '복지과 사원들이 친절하고 먹을걸 많이 준다',
 '별 욕심 없이 길게 다니기 좋은 곳  단 민원업무를 맡는다면 다름',
 '계약직으로 근무하여 무기계약직을 노리기에 아주 좋음 물론 공무원 시험합격되면 베스트임',
 '많은것을 보고 배웠음  한단계 성장하는 계기',
 '석박사 졸업하고  라인 잘 타서 편하게 살고 싶은 사람에게 추천  특히 업무가 힘들지 않아서 여성 석사 졸업자에게 추천 ',
 '새로운 실험기구를 다루고 전반적인 분석 실험 업무에 대해 배울 수 있는 곳 ',
 '공공기관 계약직이라 눈치도 보이지만 대체적으로 나쁘지는 않음 ',
 '민원인에게 과도한 친절을 베풀지 않아도 되는 곳이며  자기 할 일만 잘 하면 되는 곳임 ',
 '앞으로 더  좋아질 공기업 입니다 일은 조금 힘들다하나 저는 힘든지모르고다녔습니다',
 '다양한 무상원조 사업을 기획부터 운영 및 관리까지 경험해볼 수 있는 곳',
 '큰 목표없이 편하게 지내고 싶으신분들에게 추천합니다',
 '국가기관에서 일한다는 자부심  편리한 출퇴근 환경',
 '보수적인 분위기를 가지고 있으나 업무강도 낮은 편이며 복지 및 급여 상당히 좋은 편',
 '딱 그냥 전형적인 공무원 집단인듯 공공기관 거쳐가기 딱 좋은 계약직임',
 '기독교인이 아니어도 일할 수 있지만  YWCA에서 운영하고 있어 정규직 간사 되려면 기독교인이어야 함 ',
 '출퇴근이 편하다  회사근처에 음식점  커피먹을곳이 매무매우적다',
 '보람차게 일할 수 있는 곳  지식을 얻고자 오는 분들께 서비스를 제공할 수 있다는게 보람찼음 ',
 '그냥 저냥 공무윈입니다 다른 자치구에 비해 그닥 복지가 좋은 쪽은 아닌듯',
 '생각보다 보수적인 것 같지않아요 사람

In [35]:
okt=Okt()
okt.nouns(reviews_1[0])

['위치', '타이틀', '일도', '나름', '수']

In [36]:
#KKMA를 사용할 경우 성능이 좋지 않음. okt랑 비교해봤을 때 okt를 사용하는 편이 좋다고 판단.
kkma=Kkma()
kkma.nouns(reviews_1[0])

['위치', '타이틀', '일도', '나름', '수', '있음']

In [37]:
goverment= extend(reviews_1)
goverment

['위치',
 '타이틀',
 '일도',
 '나름',
 '수',
 '칼',
 '퇴보',
 '비교',
 '점심시간',
 '다만',
 '공무원',
 '사회',
 '보수',
 '분위기',
 '일',
 '구내',
 '식당',
 '층',
 '카페',
 '팀',
 '바이',
 '팀',
 '비교',
 '무난',
 '수',
 '곳',
 '복지',
 '사원',
 '별',
 '욕심',
 '곳',
 '단',
 '민원',
 '업무',
 '계약',
 '직',
 '근무',
 '무기계약직',
 '아주',
 '공무원',
 '시험',
 '합격',
 '베스트',
 '임',
 '보고',
 '단계',
 '계기',
 '박사',
 '졸업',
 '라인',
 '살',
 '사람',
 '추천',
 '업무',
 '여성',
 '석사',
 '졸업',
 '추천',
 '실험',
 '기구',
 '전반',
 '분석',
 '실험',
 '업무',
 '대해',
 '수',
 '곳',
 '공공기관',
 '계약',
 '직',
 '눈치',
 '보이',
 '체적',
 '민원',
 '과도',
 '친절',
 '곳',
 '자기',
 '일만',
 '곳임',
 '앞',
 '더',
 '공기업',
 '일',
 '조금',
 '저',
 '힘',
 '무상',
 '원조',
 '사업',
 '기획',
 '운영',
 '및',
 '관리',
 '경험',
 '수',
 '곳',
 '목표',
 '신분',
 '추천',
 '국가',
 '기관',
 '자부심',
 '출퇴근',
 '환경',
 '보수',
 '분위기',
 '가지',
 '업무',
 '강도',
 '편이',
 '복지',
 '및',
 '급여',
 '편',
 '그냥',
 '전형',
 '공무원',
 '집단',
 '듯',
 '공공기관',
 '계약',
 '직임',
 '기독교인',
 '일',
 '수',
 '운영',
 '정규직',
 '간사',
 '기독교인',
 '함',
 '출퇴근',
 '회사',
 '근처',
 '음식점',
 '커피',
 '곳',
 '무',
 '매우',
 '보람',
 '일',
 '수',
 '곳',
 '지식',
 '분

In [38]:
reviews_2 = sub(reviews2)
reviews_2

['독립계로 소신있는 운용을 하고 있는점 높게 평가함 판교에 위치하고 있어서 주변 환경이 좋은편임',
 '모든 문화예술공공기관이 그렇지만  사명감없이 버티기 힘든 곳',
 '실제 업무하는 사람들만 일하는 체계  나머지는 업무관련하여 신경쓰지 않으려고 함  직원들을 보호해주는 회사의 역할이 부족함 ',
 '서류작업이 많고 매우 보수적인 기업새로운 것을 시도하려하기보다 기존의 방식을 고수하는 것을 선호',
 '사원보다 대리가 훨씬많은  인사부터 복장까지 지적하는 태강학교',
 '야근 없고 휴가 자유롭게 쓰고 회식 없고 일하기 편함',
 '그냥 평범한 회사인데 발전 가능성은 애매합니다',
 '금융그룹계열사가 많이 섞여있어 관리가 미흡하다고 생각됩니다  야근이 잦고 인사평가또한 제대로 된다고 생각하지않습니다  굉장히 수직적입니다  학교보다 심한거같습니다 ',
 '일단 이 회사뿐만 아니라 손해사정업계에는 애초에 발을 들이지 않는것을 추천자기가 한 만큼 돈을 버는 인센티브제라 밥먹는 시간 아끼고 화장실가는 시간 아껴서 기계처럼 일만 하게됨 돈 많이 벌어야하니 조기출근 야근은 거의 필수스펙 딸리는데 일은 해야한다 그럼 추천 무스펙으로 입사가능대신 경력개발 안됨자기일만 잘하면되니 사회성 부족한 사람에겐 추천',
 '퇴근시간만 생각한다면 가도 좋은 회사  단  퇴근시간만',
 '분위기가 보수적이고 남성적인 편이나 여성들 업무하기에도 좋은 환경임',
 '근무환경이 쾌적하고 복지및 근무형태가 만족스러움',
 '개인의 역량은 잘만 발휘하면 급여가 오르는 직장',
 '장기근속 사원이 많은곳하지만 그런 직원들을 우습게 보는곳본인 업무외 타업무를 쉴새없이 시킴',
 '준공기업 성격의 회사  정년이 보장되어 있다고 해도 과언이 아님  공무원 마인드로 다닐 수 있다 ',
 '개인역량이 올랐을땐  그 어떤 회사와도 비교할 수 없는 인센구조  젊은 직원들이 지켜내고 있는 회사  이 부분은 정말 리스펙 ',
 '직원 복지가 잘되있으나 회사의 비용관리 노력이 부족합니다  성장가능성이 낮은 회사입니

In [39]:
bank= append(reviews_2)
bank

[['독립', '소신', '운용', '점', '평가', '함', '판교', '위치', '주변', '환경', '편임'],
 ['모든', '문화', '예술', '공공기관', '명감', '곳'],
 ['실제',
  '업무',
  '사람',
  '일',
  '체계',
  '나머지',
  '업무',
  '관련',
  '신경',
  '함',
  '직원',
  '보호',
  '회사',
  '역할',
  '부족함'],
 ['서류', '작업', '매우', '보수', '기업', '것', '시도', '기존', '방식', '고수', '것', '선호'],
 ['사원', '대리', '인사', '복장', '지적', '태강', '학교'],
 ['야근', '휴가', '회식', '일'],
 ['그냥', '회사', '발전', '가능성'],
 ['금융',
  '그룹',
  '열사',
  '관리',
  '생각',
  '야근',
  '인사',
  '평가',
  '또한',
  '제대로',
  '생각',
  '수직',
  '학교'],
 ['일단',
  '이',
  '회사',
  '손해',
  '정업',
  '애초',
  '발',
  '추천',
  '자기',
  '만큼',
  '돈',
  '버',
  '인',
  '센티',
  '브',
  '제라',
  '밥',
  '시간',
  '화장실',
  '시간',
  '기계',
  '일만',
  '돈',
  '조기',
  '출근',
  '야근',
  '거의',
  '필수',
  '스펙',
  '일',
  '추천',
  '무',
  '스펙',
  '입사',
  '가능',
  '대신',
  '경력',
  '개발',
  '기일',
  '사회',
  '사람',
  '추천'],
 ['퇴근', '시간', '생각', '가도', '회사', '단', '퇴근', '시간'],
 ['분위기', '보수', '남성', '편이', '여성', '업무', '환경', '임'],
 ['근무', '환경', '복지', '및', '근무', '형태', '움'],
 ['개인', '역량', '잘만', '발